# Preprocessing pipeline for reliability bootstrapping based on block-design-based simulation campaigns
1. Extract & cut spike trains
2. Compute filtered spike signals & mean-centering

In [ ]:
import sys
sys.path.append('../../library')

### 1. Extract & cut spike trains

Extracts (excitatory) spike trains in format compatible with "toposample" pipeline

In [ ]:
from extract import run_extraction

In [ ]:
campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_OutConnsRemoved_BlockDesign_Struct'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_OutConnsRemoved_BlockDesign_Rnd'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipConnsRemoved_BlockDesign_Struct'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipConnsRemoved_BlockDesign_Rnd'

sim_paths, working_dir = run_extraction(campaign_path, working_dir_name='working_dir_tmp')

### 2. Compute filtered spike signals & mean-centering

Runs preprocessing (filtering, mean-centering) of (excitatory) spike trains [PARALLEL IMPLEMENTATION]

In [ ]:
from preprocess import run_preprocessing, merge_into_h5_data_store

In [ ]:
# Run preprocessing
spike_file_names = [f'raw_spikes_exc_{idx}.npy' for idx in range(len(sim_paths))]

run_preprocessing(working_dir, spike_file_names, sigma=10.0, pool_size=10)

In [ ]:
# Merge individual preprocessed spike files into .h5 data store
#   split_by_gid==False ... Datasets per sim
#   split_by_gid==True  ... Datasets per sim & GID
tmp_file_names = [f'spike_signals_exc_{idx}__tmp__.npz' for idx in range(len(sim_paths))]

h5_file = merge_into_h5_data_store(working_dir, tmp_file_names, data_store_name='processed_data_store', split_by_gid=False)

__HOW TO LOAD SPIKE SIGNALS FROM .H5 DATA STORE:__
~~~
h5_store = h5py.File(h5_file)
print(f'Groups/Datasets: {list(h5_store.keys())}')
t_bins = np.array(h5_store['t_bins'])
gids = np.array(h5_store['gids'])
print(f'Spike signals per sims: {list(h5_store["spike_signals_exc"].keys())}')
if split_by_gid == True:
    print(f'Spike signals within sim <SIM_IDX>: {list(h5_store["spike_signals_exc"]["sim_<SIM_IDX>"].keys())}')
    spike_signal = np.array(h5_store['spike_signals_exc'][f'sim_{<SIM_IDX>}'][f'gid_{<GID>}'])
else:
    spike_signals = np.array(h5_store['spike_signals_exc'][f'sim_{<SIM_IDX>}'])
h5_store.close()
~~~